# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [ ]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [6]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

## Role Playing, Focus and Cooperation

In [7]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [8]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

In [11]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [12]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [13]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [14]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [15]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [16]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: I need to access the specific documentation on how to create a Crew and add memory to it. This will allow me to provide accurate and detailed guidance to Andrew Ng from DeepLearningAI.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"}
 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Activating Your CrewAI Team
 Initializing search
 crew

In [17]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew,

Thank you for reaching out! I'd be delighted to assist you with setting up a Crew and adding memory to your agents.

To create a Crew and kick it off, you'll need to follow several steps. Below, I've provided detailed guidance on how to assemble your agents, including how to add memory to them.

### Step 0: Installation
First, ensure that you have CrewAI and any necessary packages installed. CrewAI is compatible with Python versions >=3.10 and <=3.13.

```bash
pip install crewai
pip install 'crewai[tools]'
```

### Step 1: Assemble Your Agents
Define your agents with distinct roles, backstories, and enhanced capabilities. Here’s an example of how you can create agents and add memory to them:

```python
import os
from langchain.llms import OpenAI
from crewai import Agent
from crewai_tools import SerperDevTool, BrowserbaseTool, ExaSearchTool

# Set your API keys
os.environ["OPENAI_API_KEY"] = "Your OpenAI Key"
os.environ["SERPER_API_KEY"] = "Your Serper Key"

# Initialize tools
search_tool = SerperDevTool()
browser_tool = BrowserbaseTool()
exa_search_tool = ExaSearchTool()

# Creating a senior researcher agent with memory enabled
researcher = Agent(
    role='Senior Researcher',
    goal='Uncover groundbreaking technologies in {topic}',
    backstory=("Driven by curiosity, you're at the forefront of innovation, "
               "eager to explore and share knowledge that could change the world."),
    memory=True,  # Memory is enabled here
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, browser_tool],
    allow_code_execution=False,  # New attribute for enabling code execution
    max_iter=15,  # Maximum number of iterations for task execution
    max_rpm=100,  # Maximum requests per minute
    max_execution_time=3600,  # Maximum execution time in seconds
    system_template="Your custom system template here",  # Custom system template
    prompt_template="Your custom prompt template here",  # Custom prompt template
    response_template="Your custom response template here",  # Custom response template
)

# Creating a writer agent with memory enabled
writer = Agent(
    role='Writer',
    goal='Narrate compelling tech stories about {topic}',
    backstory=("With a flair for simplifying complex topics, you craft engaging "
               "narratives that captivate and educate, bringing new discoveries to light."),
    verbose=True,
    allow_delegation=False,
    memory=True,  # Memory is enabled here
    tools=[exa_search_tool],
    function_calling_llm=OpenAI(model_name="gpt-3.5-turbo"),  # Separate LLM for function calling
)

# Setting a specific manager agent
manager = Agent(
    role='Manager',
    goal='Ensure the smooth operation and coordination of the team',
    verbose=True,
    backstory=(
        "As a seasoned project manager, you excel in organizing "
        "tasks, managing timelines, and ensuring the team stays on track."
    ),
    allow_code_execution=True,  # Enable code execution for the manager
)
```

### Summary of Key Attributes
- `memory=True`: This attribute enables memory for the agent, allowing it to retain information across interactions.
- `allow_code_execution`: Enable or disable code execution capabilities for the agent (default is False).
- `max_execution_time`: Set a maximum execution time (in seconds) for the agent to complete a task.
- `function_calling_llm`: Specify a separate language model for function calling.

**Note:** The `memory_size` parameter determines how much information the agent can retain. Adjust this based on your specific needs and available resources.

### Troubleshooting and Common Errors
If you encounter any issues, consider the following troubleshooting tips:
- **Installation Issues:** Ensure that your Python environment is properly configured and that there are no conflicts with other packages.
- **Memory Errors:** If you receive memory-related errors, try reducing the `memory_size` parameter or check your system’s available resources.

For more detailed documentation, you can refer to our [official guide](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/).

If you have any more questions or need further assistance, please don't hesitate to ask. We're here to help you succeed with CrewAI!

Best regards,  
[Your Name]  
Senior Support Representative, crewAI  

**Contact Information:**  
Email: support@crewai.com  
Phone: 1-800-123-4567